# This is the code which solves the Static Traffic Assignment Problem

In [1]:
import numpy as np

In [5]:
network = 'data/I210'

## 1. We load the graph and the demand
Both graph and demand are in csv file

In [6]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [7]:
# in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])
print(demand)

[[1.9e+01 2.0e+01 2.5e+04]]


Then store the links in a dictionary

$\underline{\text{Edit:}}$ we don't need it anymore, we store the graph adjacency matrix as a sparse matrix

In [8]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    
# print(graph_dict)
# Here we remove graph_dict
del graph_dict

Then, we define the function which gives the travel time as a function of the flow

In [9]:
# TO DO ADD THE CAPACITY
def travel_time(f, c = -1):
    if c == -1:
        c = [float('inf') for i in range(len(f))] # this might be to much, to do once we have everything done
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    tt_tmp = [tt_tmp[i] if f[i]<c[i] else float('inf') for i in range(len(f))]
    return tt_tmp

In [10]:
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
print(nb_nodes)

21


## 2. We compute the all or nothing flow allocation

In [11]:
import scipy

In [12]:
from scipy.sparse.csgraph import dijkstra

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [13]:
def update_travel_time(tt):
    for i in range(graph.shape[0]):
        # G[int(graph[i][1])].getcol(int(graph[i][2])).toarray().reshape(1)[0]
        # G[int(graph[i][1])][int(graph[i][2])] = tt[i]
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]

In [14]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
update_travel_time(travel_time(np.zeros(nb_links))) # , [1,1,-1,1,1]))
# the following line make the matrix sparse, one can use G.toarray() to do the opposite
G = scipy.sparse.csr_matrix(G)
print(G)

  (1, 2)	1.38
  (1, 7)	1.15
  (2, 3)	1.38
  (2, 8)	1.14
  (3, 4)	1.68
  (3, 9)	1.13
  (4, 5)	1.26
  (4, 10)	1.05
  (5, 6)	2.25
  (5, 11)	1.04
  (6, 12)	1.09
  (7, 1)	1.15
  (7, 8)	0.48
  (7, 13)	0.36
  (8, 2)	1.14
  (8, 9)	0.48
  (8, 14)	0.39
  (9, 3)	1.13
  (9, 10)	0.58
  (9, 15)	0.38
  (10, 4)	1.05
  (10, 11)	0.43
  (10, 16)	0.48
  (11, 5)	1.04
  (11, 12)	0.77
  (11, 17)	0.5
  (12, 6)	1.09
  (12, 18)	0.47
  (12, 20)	0.22
  (13, 7)	0.36
  (13, 14)	1.38
  (14, 8)	0.39
  (14, 15)	1.38
  (15, 9)	0.38
  (15, 16)	1.68
  (16, 10)	0.48
  (16, 17)	1.25
  (17, 11)	0.5
  (17, 18)	2.22
  (18, 12)	0.47
  (19, 7)	0.29


In [15]:
class path:
    def __init__(self,links):
        self.links = links
        self.flow = 0
    def add_flow(self, flow):
        self.flow += flow
    def __eq__(self, other):
        return np.all(self.links == other.links)
    def __hash__(self):
        return hash(np.sum([hash(self.links[i]*(nb_links**i)) for i in range(len(self.links))]))
    def __str__(self):
        return str(self.flow) + " is on " + str(self.links) 

In [16]:
paths_used = {}
print(paths_used)
p = path([0, 1, 2, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)
p = path([0, 1, 1, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)
p = path([0, 0, 2, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)

{}
{210166: (<__main__.path object at 0x1080097b8>, 0)}
{210166: (<__main__.path object at 0x1080097b8>, 0), 208485: (<__main__.path object at 0x108009780>, 0)}
{210166: (<__main__.path object at 0x1080097b8>, 0), 208485: (<__main__.path object at 0x108009780>, 0), 210125: (<__main__.path object at 0x1080095c0>, 0)}


Now let's compute the all or nothing allocation

In [19]:
# computing the all or nothing flow
paths_used = {}

def all_or_nothing():
    global k
    faon = np.zeros(nb_links)
    # using scipy to compute dijkstra
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True)
    """faon = np.zeros(shape = nb_links)"""
    for i in range(nb_ods):
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]

        node_tmp = d_tmp
        links = []
        while node_tmp != o_tmp:
            node_tmp_d = return_predecessors[o_tmp][node_tmp]
            link_tmp = list(G.todok().keys()).index((node_tmp_d, node_tmp)) #I don't know how much time it takes, should I stock the list of keys in memory?
            links.insert(0, link_tmp)
            faon[link_tmp] += flow_tmp
            node_tmp = node_tmp_d
        
        p = path(links)
        p.add_flow(flow_tmp)
        print(p)
        if hash(p) in paths_used:
            continue
            # print(str(p) + " is already in paths_used" + str(paths_used[hash(p)]))
        else:
            print(k)
            paths_used[hash(p)] = (p, k)
            k = k+1
    
        # Here store the path
        # p = np.zeros()
        # next is for the link flow
        """
        node_tmp_d = d_tmp
        while node_tmp_d != o_tmp:
            node_tmp = return_predecessors[o_tmp][node_tmp_d]
            link_tmp = int(graph_dict[node_tmp][node_tmp_d])
            faon[link_tmp] += flow_tmp
            node_tmp_d = node_tmp
    return faon
    """
    return faon

We define the line search

In [20]:
k = 0 
all_or_nothing()
print(paths_used)

25000.0 is on [13, 15, 18, 21, 24, 27, 40]
0
{193201583558: (<__main__.path object at 0x108009898>, 0)}


In [21]:
# TO DO: rewrite this function
def potential(graph, f):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    links = int(np.max(graph[:, 0]) + 1)
    g = np.copy(
        graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [24]:
eps=1e-8
f = all_or_nothing()
print(f)
update_travel_time(travel_time(f))
print(paths_used)

for i in range(1000):
    if i % 100 == 0:
        print(i)
    faon = all_or_nothing() 
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
    if s < eps:
        break
    f = (1. - s) * f + s * faon
    # also update the path flow
    ## TO DO ##
    update_travel_time(travel_time(f))

25000.0 is on [13, 15, 30, 33, 19, 4, 5, 7, 9, 26, 40]
[    0.     0.     0.     0. 25000. 25000.     0. 25000.     0. 25000.
     0.     0.     0. 25000.     0. 25000.     0.     0.     0. 25000.
     0.     0.     0.     0.     0.     0. 25000.     0.     0.     0.
 25000.     0.     0. 25000.     0.     0.     0.     0.     0.     0.
 25000.]
{193201583558: (<__main__.path object at 0x108009898>, 0), 325039770805430: (<__main__.path object at 0x10817dac8>, 1), 324504196556880: (<__main__.path object at 0x10817db38>, 2), 545491554577279201: (<__main__.path object at 0x10817dc18>, 3)}
0
25000.0 is on [13, 15, 30, 33, 19, 21, 24, 27, 40]
4
25000.0 is on [13, 15, 30, 33, 19, 21, 34, 37, 39, 28, 40]
5
25000.0 is on [13, 15, 30, 33, 19, 21, 6, 7, 9, 26, 40]
6
25000.0 is on [13, 15, 30, 33, 19, 21, 34, 37, 25, 8, 9, 26, 40]
7


In [23]:
print(G)
print(f)

  (1, 2)	1.38
  (1, 7)	1.15
  (2, 3)	5089361032701062.0
  (2, 8)	4204254766144356.5
  (3, 4)	9231559.039423268
  (3, 9)	4271910701258548.0
  (4, 5)	1.26
  (4, 10)	3969474545417235.0
  (5, 6)	2.25
  (5, 11)	3931670025937070.5
  (6, 12)	1.09
  (7, 1)	1.15
  (7, 8)	0.48
  (7, 13)	3.375e+17
  (8, 2)	1.14
  (8, 9)	720000000000000.5
  (8, 14)	0.39
  (9, 3)	1.13
  (9, 10)	49434806976970.27
  (9, 15)	2088090.735107644
  (10, 4)	1.05
  (10, 11)	36649943103615.9
  (10, 16)	0.48
  (11, 5)	1.04
  (11, 12)	65628967883219.164
  (11, 17)	0.5
  (12, 6)	4120692623337891.5
  (12, 18)	2.5037025085318704e+16
  (12, 20)	1502689170669.2395
  (13, 7)	1536841197275133.8
  (13, 14)	1.38
  (14, 8)	1705160836426404.5
  (14, 15)	1.38
  (15, 9)	0.38
  (15, 16)	7345308218452204.0
  (16, 10)	0.48
  (16, 17)	5336254157205326.0
  (17, 11)	0.5
  (17, 18)	9477187383196660.0
  (18, 12)	0.47
  (19, 7)	435000000000000.3
[    0.             0.          6260.99321345  6260.99321345
    38.89901572  6299.89222917     0.      

In [25]:
print(paths_used)

{193201583558: (<__main__.path object at 0x108009898>, 0), 325039770805430: (<__main__.path object at 0x10817dac8>, 1), 324504196556880: (<__main__.path object at 0x10817db38>, 2), 545491554577279201: (<__main__.path object at 0x10817dc18>, 3), 324771866056542: (<__main__.path object at 0x10817d748>, 4), 546391848553735589: (<__main__.path object at 0x10817d6a0>, 5), 545491561296938859: (<__main__.path object at 0x10817d6d8>, 6), 1552122163731328403: (<__main__.path object at 0x10817d7f0>, 7)}
